### LS/DR9 Photometry for Fuji (EDR)

This notebook briefly shows how to work with the LS/DR9 value-added catalogs for a small set of Survey Validation (SV) targets in the Fuji production of the DESI Early Data Release (EDR). To run this notebook be sure to use the DESI kernel or, if working in a terminal, to first execute:
```
source /global/cfs/cdirs/desi/software/desi_environment.sh main
```

John Moustakas  
Siena College  

In [1]:
import os
import numpy as np
import fitsio
from astropy.table import Table, vstack

#### Specify the location of the DESI/EDR redshift catalogs and the LS/DR9 VAC.

In [2]:
dr = 'edr'
specprod = 'fuji'
lsdr9_version = 'v2.1'

zcatdir = f'/global/cfs/cdirs/desi/public/{dr}/spectro/redux/{specprod}/zcatalog'
vacdir = f'/global/cfs/cdirs/desi/public/{dr}/vac/{dr}/lsdr9-photometry/{specprod}/{lsdr9_version}/observed-targets'

#### Select a small number of LRG targets from the SV3 (One Percent Survey).

To select these targets, we apply the appropriate targeting bitmask and we also require `ZWARN==0` and `TSNR2_LRG_Z>120`, in order to keep the number of targets relatively small. (See [here](https://data.desi.lbl.gov/doc/glossary/#tsnr2) and Section 4.14 of [Guy et al. (2023)](https://ui.adsabs.harvard.edu/abs/2023AJ....165..144G/abstract) for the definition of `TSNR2`.)

In [3]:
def select_targets(survey='sv3', program='dark'):
    """Select a small number of targets."""
    if survey == 'sv1':
        from desitarget.sv1.sv1_targetmask import desi_mask
    elif survey == 'sv2':
        from desitarget.sv2.sv2_targetmask import desi_mask
    elif survey == 'sv3':
        from desitarget.sv3.sv3_targetmask import desi_mask
    else:
        raise ValueError(f'Unsupported survey {survey}!')

    zcatfile = os.path.join(zcatdir, f'ztile-{survey}-{program}-cumulative.fits')
    zcat = Table(fitsio.read(zcatfile, 'ZCATALOG'))
    I = ((zcat[f'{survey.upper()}_DESI_TARGET'] & desi_mask.mask('LRG') != 0) * 
         (zcat['ZWARN'] == 0) * (zcat['TSNR2_LRG_Z'] > 120))
    zcat = zcat[I]
    _, uindx = np.unique(zcat['TARGETID'], return_index=True)
    zcat = zcat[uindx]
    print(f'Selected {len(zcat):,d} unique LRG targets from {zcatfile}')
    return zcat

In [4]:
zcat = select_targets()
zcat

Selected 253 unique LRG targets from /global/cfs/cdirs/desi/public/edr/spectro/redux/fuji/zcatalog/ztile-sv3-dark-cumulative.fits


TARGETID,LASTNIGHT,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,PLATE_RA,PLATE_DEC,TILEID,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MEAN_PSF_TO_FIBER_SPECFLUX,MEAN_FIBER_X,MEAN_FIBER_Y,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,ZCAT_NSPEC,ZCAT_PRIMARY
int64,int32,int32,float64,float64,int64,float64,float64[10],int64,str6,str20,int64,float64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,str3,float32,float32,int32,float64,int32,int16,str8,int32,int32,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,str1,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int32,int16,float32,int16,int16,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,bool
39627740101482360,20210413,20210413,0.5248403205206574,0.00018390352387403578,0,7939.557902217843,41.40204768377702 .. 2.4269906157395216,7892,GALAXY,,10,9.908557667396963,0,419,419,303,0,220.24650202909086,-2.0734718570830664,0.0,0.0,2015.5,5400.0,65537,1,TGT,-84.56631,-381.56824,103200,0.978108886589353,1,9010,2201m020,319728,6008,EXP,0.043199174,0.80587673,2.5711763,7.083748,34.063538,29.810293,490.78506,168.7047,45.652336,2.2190568,0.568295,0.23683602,0.7556332,2.0818155,0.23897503,0.7623994,2.0942717,2048,1.0,1.5517012,0.31732282,-0.46749115,0,,0.0,0.0,0.0,0.0,S,103200,9,65537,0,0,0,0,0,0,220.24650202909086,-2.0734718570830664,249,1,909.2157,1,1,2.9535,3.410408,-1.987,2.2984421,220.25828669283496,24.511961,-2.065982811900091,15.672653,0.17909247,-87.49875,-379.47476,474.58643,0.39197996,421.13727,227.54105,9030.4375,372.10812,5.707962,7.834577,42808.676,95.303185,35374.63,0.08305293,40994.16,26276.47,15.604686,399.13864,6.1770996e-05,330.704,5.3182503e-05,0.0,67090.25,4.406155e-05,36.92303,442.61884,43283.26,426.39917,35795.766,227.6241,117114.84,26648.578,58.23568,849.59204,2,True
39627752197849929,20210413,20210413,0.7860486343586703,0.00015104179977554712,0,7894.209021531045,279.89225435880206 .. 0.06212495485499034,7883,GALAXY,,10,211.0045355334878,9,354,9354,4700,8,221.02793230291124,-1.4793354243663748,0.0,0.0,2015.5,5400.0,2162697,1,TGT,-280.43298,-229.8854,103200,0.5866609572561384,1,9010,2211m015,322612,841,DEV,0.04850275,0.7210559,3.736937,17.188488,57.625374,32.36086,965.9877,300.33807,82.53408,2.1419983,0.6286255,0.34819064,1.8045292,8.300148,0.348

#### Define a couple convenience functions.

Note: The same target can be observed on different tiles *and* in different surveys (sometimes with very different targeting information), so you have to be very careful to ensure you're matching to the correct target.

In [5]:
def read_targetphot(zcat, survey=None):
    """Read the targeting photometry for an input catalog of objects.   

    """
    from desitarget import geomask
    
    targetfile = os.path.join(vacdir, f'targetphot-{survey}-fuji.fits')
    info = fitsio.read(targetfile, 'TARGETPHOT', columns=['TARGETID', 'TILEID'])
    I = np.hstack([np.where((tileid == info['TILEID']) * (targetid == info['TARGETID']))[0]
                   for tileid, targetid in zip(zcat['TILEID'], zcat['TARGETID'])])
    targ = Table(fitsio.read(targetfile, rows=I))
    print(f'Read targeting photometry for {len(targ):,d} objects from {targetfile}')
    
    srt = geomask.match_to(targ['TARGETID'], zcat['TARGETID'])
    targ = targ[srt]

    assert(np.all(targ['TARGETID'] == zcat['TARGETID']))
    
    return targ   

In [6]:
def read_tractorphot(zcat, specprod='fuji', verbose=False):
    """Gather Tractor photometry for an input catalog. Note that this function 
    hasn't been tested very extensively and will likely fail if there are duplicate
    TARGETIDs in the input redshift catalog.
    
    """
    from glob import glob
    from desitarget import geomask
    from desimodel.footprint import radec2pix

    tractorphotfiles = glob(os.path.join(vacdir, 'tractorphot', f'tractorphot-nside4-hp???-{specprod}.fits'))
    
    hdr = fitsio.read_header(tractorphotfiles[0], 'TRACTORPHOT')
    tractorphot_nside = hdr['FILENSID']

    pixels = radec2pix(tractorphot_nside, zcat['TARGET_RA'], zcat['TARGET_DEC'])
    phot = []
    for pixel in sorted(set(pixels)):
        J = pixel == pixels
        photfile = os.path.join(vacdir, 'tractorphot', f'tractorphot-nside4-hp{pixel:03d}-{specprod}.fits')
        if os.path.isfile(photfile):
            targetids = fitsio.read(photfile, columns='TARGETID')
            K = np.where(np.isin(targetids, zcat['TARGETID'][J]))[0]

            if verbose:
                print(f'Gathering Tractor photometry for {len(K):,d} object(s) from {photfile}')

            _phot = fitsio.read(photfile, rows=K)
            phot.append(Table(_phot))

    if len(phot) > 0:
        phot = vstack(phot)
    else:
        phot = Table()

    # check for objects with missing Tractor photometry
    if len(phot) > 0:
        match = np.where(np.isin(zcat['TARGETID'], phot['TARGETID']))[0]
        miss = np.where(np.logical_not(np.isin(zcat['TARGETID'], phot['TARGETID'])))[0]

        srt = geomask.match_to(phot['TARGETID'], zcat['TARGETID'][match])
        phot = phot[srt]
        assert(np.all(phot['TARGETID'] == zcat['TARGETID'][match]))
    else:
        miss = np.arange(len(zcat))

    if len(miss) > 0:
        print(f'Missing Tractor photometry for {len(miss):,d} object(s).')
        zcat_missing = zcat[miss]
    else:
        zcat_missing = Table()

    return phot, zcat_missing

In [7]:
%time targ = read_targetphot(zcat, survey='sv3')
#targ

Read targeting photometry for 253 objects from /global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/observed-targets/targetphot-sv3-fuji.fits
CPU times: user 2.54 s, sys: 1.35 s, total: 3.89 s
Wall time: 5.25 s


In [8]:
%time tractor, zcat_missing = read_tractorphot(zcat, specprod=specprod, verbose=True)
tractor

Gathering Tractor photometry for 16 object(s) from /global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/observed-targets/tractorphot/tractorphot-nside4-hp035-fuji.fits
Gathering Tractor photometry for 6 object(s) from /global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/observed-targets/tractorphot/tractorphot-nside4-hp039-fuji.fits
Gathering Tractor photometry for 25 object(s) from /global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/observed-targets/tractorphot/tractorphot-nside4-hp040-fuji.fits
Gathering Tractor photometry for 16 object(s) from /global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/observed-targets/tractorphot/tractorphot-nside4-hp041-fuji.fits
Gathering Tractor photometry for 52 object(s) from /global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/observed-targets/tractorphot/tractorphot-nside4-hp043-fuji.fits
Gathering Tractor photometry for 3 object(s) from /global/cfs/cdirs/desi/public/e

RELEASE,BRICKID,BRICKNAME,OBJID,BRICK_PRIMARY,MASKBITS,FITBITS,TYPE,RA,DEC,RA_IVAR,DEC_IVAR,BX,BY,DCHISQ,EBV,MJD_MIN,MJD_MAX,REF_CAT,REF_ID,PMRA,PMDEC,PARALLAX,PMRA_IVAR,PMDEC_IVAR,PARALLAX_IVAR,REF_EPOCH,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_G_N_OBS,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_N_OBS,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_N_OBS,GAIA_PHOT_VARIABLE_FLAG,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_ASTROMETRIC_EXCESS_NOISE_SIG,GAIA_ASTROMETRIC_N_OBS_AL,GAIA_ASTROMETRIC_N_GOOD_OBS_AL,GAIA_ASTROMETRIC_WEIGHT_AL,GAIA_DUPLICATED_SOURCE,GAIA_A_G_VAL,GAIA_E_BP_MIN_RP_VAL,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,APFLUX_G,APFLUX_R,APFLUX_Z,APFLUX_RESID_G,APFLUX_RESID_R,APFLUX_RESID_Z,APFLUX_BLOBRESID_G,APFLUX_BLOBRESID_R,APFLUX_BLOBRESID_Z,APFLUX_IVAR_G,APFLUX_IVAR_R,APFLUX_IVAR_Z,APFLUX_MASKED_G,APFLUX_MASKED_R,APFLUX_MASKED_Z,APFLUX_W1,APFLUX_W2,APFLUX_W3,APFLUX_W4,APFLUX_RESID_W1,APFLUX_RESID_W2,APFLUX_RESID_W3,APFLUX_RESID_W4,APFLUX_IVAR_W1,APFLUX_IVAR_W2,APFLUX_IVAR_W3,APFLUX_IVAR_W4,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,NOBS_G,NOBS_R,NOBS_Z,NOBS_W1,NOBS_W2,NOBS_W3,NOBS_W4,RCHISQ_G,RCHISQ_R,RCHISQ_Z,RCHISQ_W1,RCHISQ_W2,RCHISQ_W3,RCHISQ_W4,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACFLUX_W1,FRACFLUX_W2,FRACFLUX_W3,FRACFLUX_W4,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,ANYMASK_G,ANYMASK_R,ANYMASK_Z,ALLMASK_G,ALLMASK_R,ALLMASK_Z,WISEMASK_W1,WISEMASK_W2,PSFSIZE_G,PSFSIZE_R,PSFSIZE_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,NEA_G,NEA_R,NEA_Z,BLOB_NEA_G,BLOB_NEA_R,BLOB_NEA_Z,PSFDEPTH_W1,PSFDEPTH_W2,PSFDEPTH_W3,PSFDEPTH_W4,WISE_COADD_ID,WISE_X,WISE_Y,LC_FLUX_W1,LC_FLUX_W2,LC_FLUX_IVAR_W1,LC_FLUX_IVAR_W2,LC_NOBS_W1,LC_NOBS_W2,LC_FRACFLUX_W1,LC_FRACFLUX_W2,LC_RCHISQ_W1,LC_RCHISQ_W2,LC_MJD_W1,LC_MJD_W2,LC_EPOCH_INDEX_W1,LC_EPOCH_INDEX_W2,SERSIC,SERSIC_IVAR,SHAPE_R,SHAPE_R_IVAR,SHAPE_E1,SHAPE_E1_IVAR,SHAPE_E2,SHAPE_E2_IVAR,LS_ID,TARGETID
int16,int32,str8,int32,bool,int16,int16,str3,float64,float64,float32,float32,float32,float32,float32[5],float32,float64,float64,str2,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,int16,float32,float32,int16,bool,float32,float32,int16,int16,float32,bool,float32,float32,float32,float32,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,uint8,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str8,float32,float32,float32[15],float32[15],float32[15],float32[15],int16[15],int16[15],float32[15],float32[15],float32[15],float32[15],float64[15],float64[15],int16[15],int16[15],float32,float32,float32,float32,float32,float32,float32,float32,int64,int64
9010,319728,2201m020,6008,True,2048,2,EXP,220.24650202909086,-2.0734718570830664,46932337000.0,22567342000.0,131.0957,789.8985,951.58105 ..

In this example, none of the input objects are missing Tractor photometry.

In [9]:
zcat_missing

<Table length=0>
<No columns>